In [ ]:
import cv2
import numpy as np
import time
import os
import sys
import glob

# Try different ways to import required libraries
try:
    import torch
    TORCH_AVAILABLE = True
except ImportError:
    TORCH_AVAILABLE = False
    print("Warning: PyTorch not available, will use alternative detection")

try:
    from pydobot.Dobot import MODE_PTP
    import pydobot
    ROBOT_AVAILABLE = True
except ImportError:
    ROBOT_AVAILABLE = False
    print("Warning: pydobot not available")


class DobotVisionLab:
    def __init__(self, robot_port="/dev/ttyUSB0", camera_id=None):
        """
        Initialize Dobot Vision Lab System
        """
        print("Initializing Dobot Vision Lab System...")

        # Initialize robot if available
        if ROBOT_AVAILABLE:
            self.init_robot(robot_port)
        else:
            print("Robot not available - simulation mode")
            self.device = None

        # Initialize camera
        self.init_camera(camera_id)

        # Initialize YOLO model
        self.init_yolo_model()

        # Define robot positions (calibrate these for your setup)
        self.positions = {
            'HOME': [200, 0, 100, 0],
            'PICK_UP': [230, 30, 20, 0],
            'PALLET_A_DROP': [150, 150, 50, 0],  # Food items
            'PALLET_B_DROP': [150, -150, 50, 0]  # Vehicle items
        }

        # Object classification lists
        self.food_items = [
            'apple', 'banana', 'orange', 'sandwich', 'pizza', 'donut',
            'cake', 'cookie', 'bread', 'hot dog', 'hamburger', 'carrot',
            'broccoli', 'wine glass', 'cup', 'fork', 'knife', 'spoon',
            'bowl', 'dining table', 'bottle'
        ]

        self.vehicle_items = [
            'car', 'truck', 'bus', 'motorcycle', 'bicycle', 'train',
            'boat', 'airplane', 'helicopter', 'ship'
        ]

        print("System initialized successfully")

    def init_robot(self, robot_port):
        """Initialize robot connection"""
        try:
            self.device = pydobot.Dobot(port=robot_port)
            self.device.speed(500, 500)
            print(f"Robot connected on {robot_port}")
        except Exception as e:
            print(f"Robot connection failed: {e}")
            # Try to find correct port automatically
            ports = [f"/dev/tty{p}" for p in ['USB0', 'USB1', 'ACM0', 'ACM1']]
            for port in ports:
                try:
                    if os.path.exists(port):
                        self.device = pydobot.Dobot(port=port)
                        self.device.speed(500, 500)
                        print(f"Robot connected on {port}")
                        return
                except:
                    continue
            print("Could not connect to robot")
            self.device = None

    def init_camera(self, camera_path=None):
        """Initialize robot camera connection"""
        if camera_path is None:
            video_devices = glob.glob("/dev/video*")
            print("Available cameras:", video_devices)

            for dev in video_devices:
                cap = cv2.VideoCapture(dev)
                if cap.isOpened():
                    ret, frame = cap.read()
                    if ret:
                        print(f"Using camera at {dev}")
                        self.cap = cap
                        camera_path = dev
                        break
                    cap.release()
            else:
                raise Exception("No working camera found")
        else:
            self.cap = cv2.VideoCapture(camera_path)
            if not self.cap.isOpened():
                raise Exception(f"Camera at {camera_path} not found")

        # Set camera properties
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        print(f"Robot camera initialized at {camera_path}")

    def init_yolo_model(self):
        """Initialize YOLO model with fallback options"""
        self.model = None

        if TORCH_AVAILABLE:
            try:
                print("Loading YOLO model from torch hub...")
                self.model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
                self.model.conf = 0.5
                print("YOLO model loaded successfully")
                return
            except Exception as e:
                print(f"Torch hub loading failed: {e}")

        try:
            yolo_path = os.path.join(os.getcwd(), 'yolov5')
            if os.path.exists(yolo_path):
                sys.path.append(yolo_path)
                import detect
                print("Local YOLOv5 found")
        except Exception as e:
            print(f"Local YOLO loading failed: {e}")

        print("Using fallback detection method")
        self.model = None

    def detect_objects_yolo(self, frame):
        """Detect objects using YOLO"""
        if self.model is None:
            return self.detect_objects_fallback(frame)

        try:
            results = self.model(frame)
            detections = results.pandas().xyxy[0]

            if len(detections) > 0:
                best_detection = detections.loc[detections['confidence'].idxmax()]
                object_name = best_detection['name']
                confidence = best_detection['confidence']

                x1, y1, x2, y2 = int(best_detection['xmin']), int(best_detection['ymin']), int(best_detection['xmax']), int(best_detection['ymax'])
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{object_name}: {confidence:.2f}", (x1, y1-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                if object_name.lower() in [item.lower() for item in self.food_items]:
                    classification = 'food'
                elif object_name.lower() in [item.lower() for item in self.vehicle_items]:
                    classification = 'vehicle'
                else:
                    classification = 'unknown'

                return classification, confidence, object_name, frame
            else:
                return 'unknown', 0, 'none', frame

        except Exception as e:
            print(f"YOLO detection error: {e}")
            return self.detect_objects_fallback(frame)

    def detect_objects_fallback(self, frame):
        """Simple fallback detection method using color detection"""
        print("Using fallback color-based detection")

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        red_lower = np.array([0, 50, 50])
        red_upper = np.array([10, 255, 255])
        red_mask = cv2.inRange(hsv, red_lower, red_upper)

        yellow_lower = np.array([20, 50, 50])
        yellow_upper = np.array([30, 255, 255])
        yellow_mask = cv2.inRange(hsv, yellow_lower, yellow_upper)

        red_contours, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        yellow_contours, _ = cv2.findContours(yellow_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        red_area = sum([cv2.contourArea(c) for c in red_contours])
        yellow_area = sum([cv2.contourArea(c) for c in yellow_contours])

        if red_area > 500:
            cv2.drawContours(frame, red_contours, -1, (0, 0, 255), 2)
            cv2.putText(frame, "Red Object (Food)", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            return 'food', 0.8, 'red_object', frame
        elif yellow_area > 500:
            cv2.drawContours(frame, yellow_contours, -1, (0, 255, 255), 2)
            cv2.putText(frame, "Yellow Object (Vehicle)", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            return 'vehicle', 0.8, 'yellow_object', frame
        else:
            cv2.putText(frame, "No Object Detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            return 'unknown', 0, 'none', frame

    def show_camera_feed(self, duration=10):
        print(f"Showing camera feed for {duration} seconds (press 'q' to quit)")
        start_time = time.time()
        while (time.time() - start_time) < duration:
            ret, frame = self.cap.read()
            if not ret:
                print("Failed to read from camera")
                break
            cv2.putText(frame, "Camera Feed - Press 'q' to quit", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.imshow('Camera Feed', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cv2.destroyAllWindows()

    def move_to_position(self, position_name, use_jump=False):
        if self.device is None:
            print(f"Simulating move to {position_name}")
            time.sleep(1)
            return True

        if position_name not in self.positions:
            print(f"Position {position_name} not defined")
            return False

        x, y, z, r = self.positions[position_name]
        print(f"Moving to {position_name}: ({x}, {y}, {z}, {r})")

        try:
            if use_jump:
                self.device.move_to(mode=int(MODE_PTP.JUMP_XYZ), x=x, y=y, z=z, r=r)
            else:
                self.device.move_to(mode=int(MODE_PTP.MOVJ_XYZ), x=x, y=y, z=z, r=r)
            time.sleep(2)
            return True
        except Exception as e:
            print(f"Movement error: {e}")
            return False

    def pick_up_object(self):
        print("Picking up object...")
        if self.device is None:
            print("Simulating object pickup")
            time.sleep(2)
            return True

        try:
            self.device.suck(True)
            time.sleep(1)
            current_pose, _ = self.device.get_pose()
            lift_height = current_pose[2] + 30
            self.device.move_to(mode=int(MODE_PTP.MOVJ_XYZ),
                                x=current_pose[0], y=current_pose[1],
                                z=lift_height, r=current_pose[3])
            time.sleep(1)
            return True
        except Exception as e:
            print(f"Pickup error: {e}")
            if self.device:
                self.device.suck(False)
            return False

    def drop_object(self, position_name):
        print(f"Dropping object at {position_name}")
        if self.device is None:
            print("Simulating object drop")
            time.sleep(2)
            return True

        try:
            if not self.move_to_position(position_name, use_jump=True):
                return False
            x, y, z, r = self.positions[position_name]
            drop_height = z - 15
            self.device.move_to(mode=int(MODE_PTP.MOVJ_XYZ), x=x, y=y, z=drop_height, r=r)
            time.sleep(1)
            self.device.suck(False)
            time.sleep(1)
            self.device.move_to(mode=int(MODE_PTP.MOVJ_XYZ), x=x, y=y, z=z, r=r)
            time.sleep(1)
            return True
        except Exception as e:
            print(f"Drop error: {e}")
            return False

    def run_sorting_cycle(self):
        print("\n=== Starting Sorting Cycle ===")
        print("1. Moving to pickup position...")
        if not self.move_to_position('PICK_UP'):
            return False
        print("2. Detecting object...")
        ret, frame = self.cap.read()
        if not ret:
            print("Failed to capture image")
            return False
        classification, confidence, object_name, annotated_frame = self.detect_objects_yolo(frame)
        cv2.imwrite(f"detection_{int(time.time())}.jpg", annotated_frame)
        if classification == 'unknown' or confidence < 0.3:
            print("No valid object detected")
            return False
        print(f"Detected: {object_name} -> {classification} (confidence: {confidence:.2f})")
        print("3. Picking up object...")
        if not self.pick_up_object():
            return False
        if classification == 'food':
            target = 'PALLET_A_DROP'
            print("4. Food detected -> Pallet A")
        elif classification == 'vehicle':
            target = 'PALLET_B_DROP'
            print("4. Vehicle detected -> Pallet B")
        else:
            target = 'PALLET_A_DROP'
            print("4. Unknown -> Default Pallet A")
        print("5. Dropping object...")
        if not self.drop_object(target):
            return False
        print("6. Returning home...")
        self.move_to_position('HOME', use_jump=True)
        print("=== Cycle Complete ===")
        print(f"Object: {object_name}")
        print(f"Classification: {classification}")
        print(f"Destination: {target}\n")
        return True

    def calibrate_positions(self):
        if self.device is None:
            print("Cannot calibrate without robot connection")
            return
        print("=== Position Calibration ===")
        positions = ['HOME', 'PICK_UP', 'PALLET_A_DROP', 'PALLET_B_DROP']
        for pos in positions:
            input(f"Move robot to {pos} position and press Enter...")
            pose, _ = self.device.get_pose()
            self.positions[pos] = pose
            print(f"{pos}: {pose}")
        print("\nCalibration complete! Copy these values to your code:")
        for name, pos in self.positions.items():
            print(f"'{name}': {pos},")

    def run_continuous_sorting(self, max_cycles=5):
        print("Starting continuous sorting...")
        self.move_to_position('HOME')
        cycle_count = 0
        successful_cycles = 0
        try:
            while cycle_count < max_cycles:
                print(f"\n--- Cycle {cycle_count + 1} of {max_cycles} ---")
                input("Place an object in the pickup area and press Enter...")
                success = self.run_sorting_cycle()
                cycle_count += 1
                if success:
                    successful_cycles += 1
                    print(f"Cycle {cycle_count} successful!")
                else:
                    print(f"Cycle {cycle_count} failed!")
                time.sleep(2)
        except KeyboardInterrupt:
            print("\nStopping...")
        print(f"\n=== Summary ===")
        print(f"Total cycles: {cycle_count}")
        print(f"Successful: {successful_cycles}")
        print(f"Success rate: {(successful_cycles/cycle_count)*100:.1f}%")
        self.cleanup()

    def cleanup(self):
        print("Cleaning up...")
        if hasattr(self, 'cap'):
            self.cap.release()
        if self.device:
            self.device.suck(False)
            self.device.close()
        cv2.destroyAllWindows()


def main():
    print("Dobot Vision Lab 3")
    print("Food vs Vehicle Sorting")
    print("=" * 30)

    robot_port = "/dev/ttyUSB0"
    for port in ["/dev/ttyUSB0", "/dev/ttyUSB1", "/dev/ttyACM0", "/dev/ttyACM1"]:
        if os.path.exists(port):
            robot_port = port
            break

    try:
        lab_system = DobotVisionLab(robot_port=robot_port, camera_id=None)
        while True:
            print("\nSelect option:")
            print("1. Test camera feed")
            print("2. Calibrate positions")
            print("3. Run single cycle")
            print("4. Run continuous sorting")
            print("5. Exit")
            choice = input("Enter choice (1-5): ").strip()
            if choice == '1':
                lab_system.show_camera_feed(30)
            elif choice == '2':
                lab_system.calibrate_positions()
            elif choice == '3':
                success = lab_system.run_sorting_cycle()
                print("Success!" if success else "Failed!")
            elif choice == '4':
                cycles = input("Number of cycles (default 5): ").strip()
                max_cycles = int(cycles) if cycles.isdigit() else 5
                lab_system.run_continuous_sorting(max_cycles)
            elif choice == '5':
                break
            else:
                print("Invalid choice")
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
    finally:
        if 'lab_system' in locals():
            lab_system.cleanup()


if __name__ == "__main__":
    main()
